In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
# test generation of simulated bar-like pattern

RF = np.zeros((32,32))
N = 32
x, y = np.meshgrid(np.arange(N),np.arange(N))
RF[np.sqrt((x-N/2)**2 + 8*(y-N/2)**2)<20] = -1
print(RF.shape)
plt.figure()
plt.imshow(RF)

In [ ]:
N_orient = 50
N_widths = 30
N_phases = 30
from scipy.ndimage.filters import convolve

# finds bar-like receptive field that is most similar to the given RF, outputs the similarity
# to this bar at all orientations.  The inhomogeneity in these similarity scores
# is an approximate measure of how oriented the given RF is
def QuantifyOrientedness(RF, N=32):

    x, y = np.meshgrid(np.arange(N),np.arange(N))


    S = np.zeros((N_orient,N_widths,N_phases))


    alphas = 2*np.pi * np.linspace(0,1 - 1 / N_orient,N_orient)
    widths = 2*np.round(np.logspace(1,np.log(N/4)/np.log(2), N_widths,base = 2))
    phases = np.arange(N_phases)

    # convolve the filter with a bank of bars with all widths, orientations, phases
    for a in np.arange(len(alphas)):
        for w in np.arange(len(widths)):
            for p in np.arange(len(phases)):
                alpha = alphas[a]
                width = widths[w]
                phase = phases[p]
                #G = ((np.cos(alpha)*(x+ phase)+ np.sin(alpha)*(y+ phase)) % width >= width/2)-0.5 #+ (y % width > width/2)
                n = (np.cos(alpha)*(x- phase)+ np.sin(alpha)*(y- phase))
                G = (n >= 0)*(n <= width)
                S[a,w,p] += np.abs(np.mean((G- np.mean(G)) * (RF- np.mean(RF))))#np.sum(RF*G) S[a,w,p] = np.abs(np.sum(RF*G))#

    # find the preferred bar width among all stimuli
    #print(len(np.argwhere(S==S.max())))
    smooth_size = 5
    S = np.reshape(S, [2, N_orient//2, N_widths, N_phases])
    S = np.max(S, axis = 0)
    S = np.concatenate([S, S], axis=0)
    S = np.concatenate([S, S], axis=0)
    S = convolve(S, np.ones([smooth_size, smooth_size, smooth_size]))[smooth_size:N_orient+smooth_size]
    [a, w, p] = np.argwhere(S==S.max())[0]
    alpha = alphas[a]
    width = widths[w]
    phase = phases[p]





    best_width = w
    kS = S[:,best_width,:]
    kS.shape


    # for each rotation of this bar with the best bar width, what is the best response at the preferred phase
    scores = np.max(kS,axis = 1)

    scores = np.concatenate([scores[a:], scores[:a]], axis=0)
    scores = scores / np.max(scores)
    return alpha, width, phase, scores


alpha, width, phase, scores = QuantifyOrientedness(RF)
alphas = 2*np.pi * np.linspace(0,1 - 1 / N_orient,N_orient)
print(alpha)
n = (np.cos(alpha)*(x- phase)+ np.sin(alpha)*(y- phase))
G = (n >= 0)*(n <= width)
# plot the preferred stimuli with the preferred orientation, phase and bar width overlayed with the RF
plt.figure()
plt.imshow(G)
# plot this score
plt.figure()
ax = plt.subplot(111, projection='polar')
ax.plot(alphas, scores, 'o')
ax.set_theta_zero_location("S")


In [ ]:
#compute orientedness scores for RFs 
results_dict = {}
import os

noise_start = 0.0
noise_end = 0.0
retina_out_weight_reg = 0.0
retina_out_stride = 1
retina_hidden_channels = 32
task = 'classification'
filter_size = 9
retina_layers = 2
use_b = 1
actreg = 0.0
vvs_width = 32
epochs = 20
reg = 0.0
num_trials = 10
load_dir = os.path.join(os.getcwd(), 'saved_filters')

for brain_layers in [2]:
    for retina_width in [1, 32]:
      for layer_name in ['vvs_1', 'retina_2']:
        count = 0
        scores_total = []
        for trial in range(1, 1+num_trials):
            trial_label = 'Trial'+str(trial)
            retina_out_width = retina_width
            vvs_layers = brain_layers
            model_name = 'cifar10_type_'+trial_label+'_noise_start_'+str(noise_start)+'_noise_end_'+str(noise_end)+'_reg_'+str(reg)+'_retina_reg_'+str(retina_out_weight_reg)+'_retina_hidden_channels_'+str(retina_hidden_channels)+'_SS_'+str(retina_out_stride)+'_task_'+task+'_filter_size_'+str(filter_size)+'_retina_layers_'+str(retina_layers)+'_vvs_layers'+str(vvs_layers)+'_bias_'+str(use_b)+'_actreg_'+str(actreg)+'_retina_out_channels_'+str(retina_out_width)+'_vvs_width_'+str(vvs_width)+'_epochs_'+str(epochs)
            model_name = 'SAVED'+'_'+model_name
            filename = 'FIL_'+model_name+'_'+str(layer_name)+'.npy'
            file_path = os.path.join(load_dir, filename)
            RFs = np.load(file_path)
            for RF_index in range(RFs.shape[0]):
                print(retina_width, trial, RF_index)
                RF = np.squeeze(RFs[RF_index])
                alpha, width, phase, scores = QuantifyOrientedness(RF)
                if not np.isnan(np.mean(scores)):
                    scores_total.append(scores)

                #alphas = 2*np.pi * np.linspace(0,1,N_orient)
                # plot this score
                #plt.figure()
                #ax = plt.subplot(111, projection='polar')
                #ax.plot(alphas, scores, '-o')
                #ax.set_theta_zero_location("S")
                #plt.show()
        results_dict[(brain_layers, retina_width, layer_name)] = scores_total
    np.save('orientedness_results_new.npy', results_dict)

In [ ]:
#Show polar plot visualization of RF isotropy (avg)

for brain_layers in [2]:

    for retina_width in [1, 32]:
      for layer_name in ['vvs_1', 'retina_2']: 
        results = np.array(results_dict[(brain_layers, retina_width, layer_name)])
        print(results.shape)
        results = np.concatenate([results, results[:, 0:1]], axis=1)
        avg_scores = np.mean(results, axis=0)
        err_scores = 1.96 * np.std(results, axis=0) / np.sqrt(results.shape[0])
        alphas = 2*np.pi * np.linspace(0,1 - 1 / N_orient,N_orient)
        alphas = np.concatenate([alphas, alphas[0:1]], axis=0)

        # plot this score
        plt.figure()
        ax = plt.subplot(111, projection='polar')
        ax.plot(alphas, avg_scores, '-o')
        ax.errorbar(alphas, avg_scores, yerr=err_scores,  capsize=0)
        ax.set_theta_zero_location("S")
        plt.savefig('Quantify_Orientedness_'+str(brain_layers)+'_'+str(retina_width)+'_'+str(layer_name)+'.eps', format='eps')


In [ ]:
#Show polar plot visualization of RF isotropy (samples)


for brain_layers in [2]:

    for retina_width in [1, 32]:
      for layer_name in ['vvs_1', 'retina_2']:
        plt.figure()
        ax = plt.subplot(111, projection='polar')
        results = np.array(results_dict[(brain_layers, retina_width, layer_name)])
        results = np.concatenate([results, results[:, 0:1]], axis=1)
        samples = np.random.choice(range(results.shape[0]), size=[10])

        for trial in np.random.choice(range(results.shape[0]), size=[10]):
            scores = results[trial]
            alphas = 2*np.pi * np.linspace(0,1,N_orient)
            alphas = np.concatenate([alphas, alphas[0:1]], axis=0)
            ax.plot(alphas, scores, '-')
            #ax.errorbar(alphas, avg_scores, yerr=err_scores,  capsize=0)
            ax.set_theta_zero_location("S")
            plt.savefig('samples_Quantify_Orientedness_'+str(brain_layers)+'_'+str(retina_width)+'_'+str(layer_name)+'.eps', format='eps')


In [ ]:
#orientedness quantification 1 (preference for preferred direction vs. orthogonal direction)

for brain_layers in [2]:

    for retina_width in [1, 32]:
      for layer_name in ['vvs_1', 'retina_2']:
        results = np.array(results_dict[(brain_layers, retina_width, layer_name)])
        results = np.concatenate([results, results[:, 0:1]], axis=1)
        orientedness_scores = results[:, 0] / results[:, 13]
        avg_orientedness_scores = np.mean(orientedness_scores, axis=0)
        err_scores = 1.96 * np.std(orientedness_scores, axis=0) / np.sqrt(len(orientedness_scores))
        print(retina_width, layer_name, ': ', avg_orientedness_scores, err_scores)


In [ ]:
#orientedness quantification 2 (std in preferences across all directions)


for brain_layers in [2]:

    for retina_width in [1, 32]:
      for layer in ['retina_2', 'vvs_1']:
        results = np.array(results_dict[(brain_layers, retina_width, layer)])

        std_results = np.std(results, axis=1)
        print(retina_width, layer_name, ': ', np.mean(std_results), 'plus or minus', 1.96 * np.std(std_results) / np.sqrt(len(std_results)))

In [ ]:
#Score random RFs for random baseline comparison

randscores = []
for i in range(100):
    print(i)
    _, _, _, randscore = QuantifyOrientedness(np.random.randn(32, 32))
    randscores.append(randscore)



In [ ]:
#Orientedness quantification 1 for random RFs
randscores = np.array(randscores)
orientedness_scores = randscores[:, 0] / randscores[:, 13]
avg_orientedness_scores = np.mean(orientedness_scores, axis=0)
err_scores = 1.96 * np.std(orientedness_scores, axis=0) / np.sqrt(len(orientedness_scores))
print(avg_orientedness_scores, 'plus or minus', err_scores)


In [ ]:
#Orientedness quantification 2 for random RFs

randscores = np.array(randscores)
std_rand = np.std(randscores, axis=1)
std_bn = np.std(results_dict[(2, 1, 'retina_2')], axis=1)

from scipy.stats import ttest_ind
print(np.mean(std_rand), 'plus or minus', 1.96 * np.std(std_rand) / np.sqrt(len(std_rand)))

In [ ]:
#Compute orientedness of V1 weights

load_dir = os.path.join(os.getcwd(), 'saved_weights')

noise_start = 0.0
noise_end = 0.0
retina_out_weight_reg = 0.0
retina_out_stride = 1
retina_hidden_channels = 32
task = 'classification'
filter_size = 9
retina_layers = 2
use_b = 1
actreg = 0.0
vvs_width = 32
epochs = 20
reg = 0.0
num_trials = 10
brain_layers = 2
retina_width = 2
layer = 3

scores_total = []

for trial in range(1, 1+num_trials):
    trial_label = 'Trial'+str(trial)
    retina_out_width = retina_width
    vvs_layers = brain_layers
    model_name = 'cifar10_type_'+trial_label+'_noise_start_'+str(noise_start)+'_noise_end_'+str(noise_end)+'_reg_'+str(reg)+'_retina_reg_'+str(retina_out_weight_reg)+'_retina_hidden_channels_'+str(retina_hidden_channels)+'_SS_'+str(retina_out_stride)+'_task_'+task+'_filter_size_'+str(filter_size)+'_retina_layers_'+str(retina_layers)+'_vvs_layers'+str(vvs_layers)+'_bias_'+str(use_b)+'_actreg_'+str(actreg)+'_retina_out_channels_'+str(retina_out_width)+'_vvs_width_'+str(vvs_width)+'_epochs_'+str(epochs)
    model_name = 'SAVED'+'_'+model_name
    filename = 'V1W_'+model_name+'.npy'
    file_path = os.path.join(load_dir, filename)
    W = np.load(file_path)[0]
    for in_channel in range(W.shape[2]):
        for out_channel in range(W.shape[3]):
            print(trial, in_channel, out_channel)
            filt = W[:, :, in_channel, out_channel]
            alpha, width, phase, scores = QuantifyOrientedness(filt, N=9)
            scores_total.append(scores)
            
            
scores_total = np.array(scores_total)

In [ ]:
#Show polar plot visualization of V1 weight isotropy (avg)


import matplotlib.pyplot as plt
results = np.array(scores_total)
results = np.concatenate([results, results[:, 0:1]], axis=1)
avg_scores = np.mean(results, axis=0)
err_scores = 1.96 * np.std(results, axis=0) / np.sqrt(results.shape[0])
alphas = 2*np.pi * np.linspace(0,1 - 1 / N_orient,N_orient)
alphas = np.concatenate([alphas, alphas[0:1]], axis=0)

# plot this score
plt.figure()
ax = plt.subplot(111, projection='polar')
ax.plot(alphas, avg_scores, '-o')
ax.errorbar(alphas, avg_scores, yerr=err_scores,  capsize=0)
ax.set_theta_zero_location("S")
plt.savefig('Quantfy_HW_avg.eps', format='eps')
plt.show()

In [ ]:
#Show polar plot visualization of V1 weight isotropy (samples)


plt.figure()
ax = plt.subplot(111, projection='polar')

results = np.array(scores_total)
results = np.concatenate([results, results[:, 0:1]], axis=1)
samples = np.random.choice(range(results.shape[0]), size=[10])
for trial in np.random.choice(range(results.shape[0]), size=[20]):

    scores = results[trial]
    alphas = 2*np.pi * np.linspace(0,1,N_orient)
    alphas = np.concatenate([alphas, alphas[0:1]], axis=0)
    ax.plot(alphas, scores, '-')
    #ax.errorbar(alphas, avg_scores, yerr=err_scores,  capsize=0)
    ax.set_theta_zero_location("S")
    
plt.savefig('Quantfy_HW_sample.eps', format='eps')
plt.show()